# Define the Model

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

class LyricsGeneratorModel(nn.Module):
    def __init__(self, vocab_size, num_genres, num_numeric_features):
        super(LyricsGeneratorModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, 128)
        self.lstm = nn.LSTM(128, 128, batch_first=True)
        self.dense_numeric = nn.Linear(num_numeric_features, 32)
        self.dense_genre = nn.Linear(num_genres, 32)
        self.dense_combined = nn.Linear(128 + 32 + 32, 128)
        self.output_layer = nn.Linear(128, vocab_size)

    def forward(self, lyrics_input, numeric_input, genre_input):
        embedded_lyrics = self.embedding(lyrics_input)
        lstm_out, _ = self.lstm(embedded_lyrics)
        lstm_out = lstm_out[:, -1, :]  # Get the output of the last LSTM cell

        numeric_out = F.relu(self.dense_numeric(numeric_input))
        genre_out = F.relu(self.dense_genre(genre_input))

        combined = torch.cat((lstm_out, numeric_out, genre_out), dim=1)
        combined = F.relu(self.dense_combined(combined))
        output = self.output_layer(combined)
        return output


# Dataloader

In [ ]:
class LyricsDataset(Dataset):
    def __init__(self, lyrics_sequences, numeric_features, genre_features, labels):
        self.lyrics_sequences = lyrics_sequences
        self.numeric_features = numeric_features
        self.genre_features = genre_features
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.lyrics_sequences[idx], self.numeric_features[idx], self.genre_features[idx], self.labels[idx]

# Convert your preprocessed data to PyTorch tensors and create a dataset
train_dataset = LyricsDataset(X_train_seq, X_train_num, X_train_cat, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)


# Training

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = LyricsGeneratorModel(vocab_size, num_genres, num_numeric_features).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(num_epochs):
    model.train()
    for lyrics, numeric, genre, labels in train_loader:
        lyrics, numeric, genre, labels = lyrics.to(device), numeric.to(device), genre.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(lyrics, numeric, genre)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')


In [ ]:
def generate_lyrics(model, seed_text, tokenizer, max_sequence_len=100, generation_length=50):
    model.eval()  # Set the model to evaluation mode
    generated_text = seed_text
    vocab_size = len(tokenizer.word_index) + 1  # Plus one for the zero padding

    for _ in range(generation_length):
        # Tokenize and pad the current sequence
        token_list = tokenizer.texts_to_sequences([generated_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len, padding='pre')
        token_list = torch.tensor(token_list, dtype=torch.long).to(device)

        # Predict the next word (as a token)
        with torch.no_grad():
            predictions = model(token_list).cpu()
        predicted_token = torch.argmax(predictions, dim=1)[-1].item()

        # Find the word corresponding to the predicted token
        predicted_word = ''
        for word, index in tokenizer.word_index.items():
            if index == predicted_token:
                predicted_word = word
                break

        # Append the predicted word to the generated text
        generated_text += " " + predicted_word

    return generated_text

lyrics = generate_lyrics(model, 'The sun shines', tokenizer)
print(lyrics)

features to consider:

- Think about beam search
- try many lstm layers